In [19]:
import geopandas as gpd
import pandas as pd
from osgeo import gdal, ogr
from shapely.geometry import box
import os

# Chemins des fichiers
input_shp = "../../data/project/FORMATION_VEGETALE.shp"  # Shapefile d'entrée
input_raster = "../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif"  # Raster d'entrée
output_shp = "../results/data/sample/Sample_BD_foret_T31TCJ.shp"  # Shapefile de sortie
temp_raster = "../results/data/sample/temp_reprojected.tif"  # Raster temporaire

# Charger le shapefile
gdf = gpd.read_file(input_shp)

# Attributs TFV à traiter et leurs correspondances
mapping = {
    "Forêt fermée d’un autre feuillu pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de châtaignier pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de hêtre pur": ("Autres Feuillus", 11, "Autres feuillus", 11),
    "Forêt fermée de chênes décidus purs": ("Chêne", 12, "Chêne", 12),
    "Forêt fermée de robinier pur": ("Robinier", 13, "Robinier", 13),
    "Peuplerai": ("Peupleraie", 14, "Peupleraie", 14),
    "Forêt fermée à mélange de feuillus": (None, None, "Mélange de feuillus", 15),
    "Forêt fermée de feuillus purs en îlots": (None, None, "feuillus en îlots", 16),
    "Forêt fermée d’un autre conifère pur autre que pin": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée de mélèze pur": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée de sapin ou épicéa": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée à mélange d’autres conifères": ("Autres conifères autre que pin", 21, "Autres conifères autre que pin", 21),
    "Forêt fermée d’un autre pin pur": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée de pin sylvestre pur": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée à mélange de pins purs": ("Autres pin", 22, "Autres pin", 22),
    "Forêt fermée de douglas pur": ("Douglas", 23, "Douglas", 23),
    "Forêt fermée de pin laricio ou pin noir pur": ("Pin laricio ou pin noir", 24, "Pin laricio ou pin noir", 24),
    "Forêt fermée de pin maritime pur": ("Pin maritime", 25, "Pin maritime", 25),
    "Forêt fermée à mélange de conifères": (None, None, "Mélange conifères", 26),
    "Forêt fermée de conifères en îlots": (None, None, "Conifères en îlots", 27),
    "Forêt fermée à mélange de conifères prépondérants et feuillus": (None, None, "Mélange de conifères prépondérants et feuillus", 28),
    "Forêt fermée à mélange de feuillus prépondérants et conifères": (None, None, "Mélange de feuillus prépondérants et conifères", 29),
}

In [20]:
# Ajouter les nouveaux champs "Nom" et "Code" (2 fois pour pixel et objet)
def map_tfv(tfv):
    return mapping.get(tfv, (None, None, None, None))  # Toujours retourner 4 éléments

# Vérifiez que la colonne "TFV" existe
if "TFV" in gdf.columns:
    # Appliquer le mappage et créer les colonnes
    mapped_values = gdf["TFV"].apply(map_tfv)
    mapped_df = pd.DataFrame(mapped_values.tolist(), columns=["Nom_pixel", "Code_pixel", "Nom_objet", "Code_objet"])
    
    # Ajouter les colonnes mappées au GeoDataFrame
    gdf = pd.concat([gdf, mapped_df], axis=1)
    
    # Supprimer les objets dont la valeur de "TFV" n'est pas dans le dictionnaire mapping
    gdf = gdf[gdf["TFV"].isin(mapping.keys())]
    
    print("Les colonnes 'Nom_pixel', 'Code_pixel', 'Nom_objet', et 'Code_objet' ont été créées avec succès.")
else:
    raise ValueError("Erreur : La colonne 'TFV' est absente du GeoDataFrame. Impossible de créer les nouvelles colonnes.")

# Sauvegarder le fichier shapefile temporaire
gdf.to_file(output_shp, driver="ESRI Shapefile")
print(f"Fichier shapefile temporaire créé : {output_shp}")

Les colonnes 'Nom_pixel', 'Code_pixel', 'Nom_objet', et 'Code_objet' ont été créées avec succès.
Fichier shapefile temporaire créé : ../results/data/sample/Sample_BD_foret_T31TCJ.shp


In [21]:
# Charger le raster avec GDAL
raster = gdal.Open(input_raster)

# Récupérer le système de coordonnées du shapefile (EPSG:2154)
shp_crs = gdf.crs.to_string()

# Reprojection du raster
gdal.Warp(temp_raster, raster, dstSRS=shp_crs)
print(f"Raster temporaire reprojeté créé : {temp_raster}")

Raster temporaire reprojeté créé : ../results/data/sample/temp_reprojected.tif


In [22]:
# Charger le raster reprojeté
raster_reprojected = gdal.Open(temp_raster)

# Découper le shapefile par les limites du raster
transform = raster_reprojected.GetGeoTransform()
xmin = transform[0]
xmax = xmin + transform[1] * raster_reprojected.RasterXSize
ymax = transform[3]
ymin = ymax + transform[5] * raster_reprojected.RasterYSize
raster_bounds = box(xmin, ymin, xmax, ymax)

# Charger le shapefile temporaire et effectuer la découpe
gdf_temp = gpd.read_file(output_shp)
gdf_clipped = gdf_temp[gdf_temp.geometry.intersects(raster_bounds)]

# Sauvegarder le fichier shapefile final
gdf_clipped.to_file(output_shp, driver="ESRI Shapefile")
print(f"Shapefile découpé créé : {output_shp}")

Shapefile découpé créé : ../results/data/sample/Sample_BD_foret_T31TCJ.shp


In [23]:
# Nettoyer les fichiers temporaires
if os.path.exists(temp_raster):
    os.remove(temp_raster)

print("Fichiers temporaires supprimés.")

Fichiers temporaires supprimés.
